# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy==2.2.0 allennlp==0.9.0
import spacy
print(spacy.__version__)

     |████████████████████████████████| 10.2MB 3.4MB/s 
     |████████████████████████████████| 7.6MB 29.2MB/s 
     |████████████████████████████████| 2.1MB 49.3MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 133kB 58.2MB/s 
     |████████████████████████████████| 163kB 58.6MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 5.3MB 38.8MB/s 
     |████████████████████████████████| 266kB 52.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 245kB 54.2MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 317kB 55.2MB/s 
     |████████████████████████████████| 7.1MB 48.7MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 1.1MB 50.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 245kB 57.5MB/s 


In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [3]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

159377B [00:00, 2405548.94B/s]
562it [00:00, 18351.67it/s]
8366B [00:00, 2868668.02B/s]
23it [00:00, 6883.76it/s]
100%|██████████| 585/585 [00:00<00:00, 62635.11it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9020 ||: 100%|██████████| 281/281 [00:01<00:00, 173.01it/s]
accuracy: 0.7878, loss: 1.2090 ||: 100%|██████████| 12/12 [00:00<00:00, 350.75it/s]
accuracy: 0.8442, loss: 0.7302 ||: 100%|██████████| 281/281 [00:01<00:00, 199.52it/s]
accuracy: 0.7878, loss: 1.1959 ||: 100%|██████████| 12/12 [00:00<00:00, 389.33it/s]
accuracy: 0.8442, loss: 0.7162 ||: 100%|██████████| 281/281 [00:01<00:00, 200.64it/s]
accuracy: 0.7878, loss: 1.1849 ||: 100%|██████████| 12/12 [00:00<00:00, 309.62it/s]
accuracy: 0.8442, loss: 0.7071 ||: 100%|██████████| 281/281 [00:01<00:00, 203.63it/s]
accuracy: 0.7878, loss: 1.1789 ||: 100%|██████████| 12/12 [00:00<00:00, 388.85it/s]
accuracy: 0.8442, loss: 0.6988 ||: 100%|██████████| 281/281 [00:01<00:00, 199.38it/s]
accuracy: 0.7878, loss: 1.1625 ||: 100%|██████████| 12/12 [00:00<00:00, 382.79it/s]
accuracy: 0.8442, loss: 0.6905 ||: 100%|██████████| 281/281 [00:01<00:00, 194.98it/s]
accuracy: 0.7878, loss: 1.1497 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8734693877551021,
 'best_validation_loss': 0.40723781945416704,
 'epoch': 99,
 'peak_cpu_memory_MB': 3825.408,
 'peak_gpu_0_memory_MB': 957,
 'training_accuracy': 0.9228895244147866,
 'training_cpu_memory_MB': 3825.408,
 'training_duration': '0:02:32.427415',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 957,
 'training_loss': 0.20239664090953077,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8734693877551021,
 'validation_loss': 0.40723781945416704}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [6]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [7]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for i in range(len(tagged)):
    for j in range(len(tagged[i])):
      if(tagged[i][j-1][2] == 'O' and tagged[i][j][2][0:2] == 'I-'):
        count=count+1
  
      if(tagged[i][j-1][2][0:2] == 'B-' and tagged[i][j][2][0:2] == 'I-'):
        if(tagged[i][j-1][2][2:] != tagged[i][j][2][2:]):
          count=count+1
    
      if(tagged[i][j-1][2][0:2] == 'I-' and tagged[i][j][2][0:2] == 'I-'):
        if(tagged[i][j-1][2][2:] != tagged[i][j][2][2:]):
          count=count+1
  
      if(j == 0 and tagged[i][j][2][0:2] == 'I-'):
        count=count+1
  return count


# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# Take tagger output as input

def span_stats(tagged):
  output_Pred = [[word[2] for word in lst] for lst in tagged]
  output_GS= [[word[1] for word in lst] for lst in tagged]
  span_Actual = []
  span_Pred= []

  for tag_GS in output_GS:
    i = 0
    spans_GS = []
    while i < len(tag_GS):
      if tag_GS[i][0] == "B":
        first, second = tag_GS[i].split("-")
        j = i+1
        start = i
        end = i
        while j < len(tag_GS):
          if tag_GS[j][0] == "I":
            if tag_GS[j].split("-")[1] == second:
              end = j
              j=j+1
            else:
              break
          else:
            break
        spans_GS.append((second, start, end))
        i = j
      else:
        i=i+1
    span_Actual.append(spans_GS)

  for tag_Pred in output_Pred:
    i = 0
    spans_P = []
    while i < len(tag_Pred):
      if tag_Pred[i][0] == "B":
        first, second = tag_Pred[i].split("-")
        start = i
        end = i
        j=i+1
        while j < len(tag_Pred):
          if tag_Pred[j][0] == "I":
            if tag_Pred[j].split("-")[1] == second:
              end = j
              j=j+1
            else:
              break
          else:
            break
        spans_P.append((second, start, end))
        i = j
      else:
        i=i+1
    span_Pred.append(spans_P)


  len_Actual = 0
  len_Pred = 0
  for i in span_Actual:
    len_Actual=len_Actual+len(i)
  for j in span_Pred:
    len_Pred=len_Pred+len(j)
  span_T = 0
  for i in range(len(span_Pred)):
    for j in range(len(span_Pred[i])):
      if span_Pred[i][j] in span_Actual[i]:
        span_T= span_T+1


  precision = span_T/len_Pred
  recall = span_T/len_Actual
  f1 = (2 * precision * recall) / (precision + recall)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
print(violations(baseline_output))
print(span_stats(baseline_output))

37
{'precision': 0.27586206896551724, 'recall': 0.18604651162790697, 'f1': 0.2222222222222222}


## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [8]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))



{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [9]:
# Viterbi Decoder
def viterbiDecoder(input_Sentences):
  predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

  labels = vocab.get_index_to_token_vocabulary('labels')
  labels_U = list(labels.values())

  for s in input_Sentences:
    emission_Scores = predictor.predict_instance(s)['tag_logits']
    emission_Prob = pd.DataFrame(emission_Scores, index = [token.text for token in list(s['tokens'])], columns = labels_U)

  # Matrix
  transition_Prob = pd.DataFrame(index=labels.values(),columns=labels.values())
  transition_Prob = transition_Prob.fillna(0)  
  for i in range(len(transition_Prob)):
    for j in range(len(transition_Prob)):
      if labels_U[j] != 'O':
        previous = labels_U[i][0]
        current = labels_U[j][0]
        labelType_Previous= labels_U[i][2:]
        labelType_Current= labels_U[j][2:]  
        if(current == 'I' and previous == 'O'):  
          transition_Prob.iloc[i,j] = float('-inf')
        elif(current =='I' and previous == 'I' and labelType_Previous != labelType_Current): 
          transition_Prob.iloc[i,j] = float('-inf')
        elif(current == 'I' and previous == 'B' and labelType_Previous != labelType_Current): 
          transition_Prob.iloc[i,j] = float('-inf')    
        else:
          continue  

  prob_initial = [0.0] * len(labels_U)
  tags = path_DP(prob_initial, emission_Prob, transition_Prob)
  tags_Sentence = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tags])
  tags_Total = []
  tags_Total.append(list(tags_Sentence))
  return tags_Total

#DP path
def path_DP(initial, emission, transition):
  tokens_N = emission.shape[0]
  labels_N = len(initial)
  m1 = np.array([[0.0 for j in range(tokens_N)] for i in range(labels_N)])
  m2 = np.array([[-1 for j in range(tokens_N)] for i in range(labels_N)])
  for i in range(tokens_N):
    if i == 0:
      m1[:,0] = np.add(initial, emission.iloc[i])
    else:
      for j in range(labels_N):
        m1[j,i]= np.max(np.add(transition.iloc[:,j] , m1[:,i-1])) + emission.iloc[i,j]
        m2[j,i] = np.argmax(np.add(transition.iloc[:,j] , m1[:,i-1]))
  
  score = np.max(m1[:,i])
  index = np.argmax(m1[:,i])
  
  lst = []
  while index != -1:
    lst.append(index)
    index = m2[index,i]
    i=i-1
  lst.reverse()
  return(lst)

In [10]:
output_Viterbi = viterbiDecoder(validation_dataset)
#Run your violations function on the output of this decoder to make sure that there are no invalid tag transitions
print("Violations:",violations(output_Viterbi))
#compare the span-level metrics on baseline_output and your new output using your span_stats function
print("Span stats:",span_stats(output_Viterbi))

Violations: 0
Span stats: {'precision': 0.5, 'recall': 0.5, 'f1': 0.5}
